# Wildfire Fire Detection from Satellite Imagery 

## Imported Libraries

In [1]:
#Import Libraries
import os
import sys
import time
import pandas as pd
import numpy as np
import cv2 
import matplotlib.pyplot as plt

In [2]:
# Add the path to the functions directory
sys.path.append('../functions')  # Add the path to the functions directory

## 1. Load Dataset (Satellite Imagery)

### A. Image path specification

In [3]:
# import user defined function for loading image from /functions 
from img_read import load_image

In [4]:
#Specify Image path
imagePath = 'images/fire.jpg'

### B. Load image using loadImg(imagePath)

In [5]:
image = load_image(imagePath)

NameError: name 'time' is not defined

In [ ]:
image.dtype

### C. Image dimention

In [ ]:
## Image shape
print(f"Image shape : {image.shape}")

### D. View Image matrix

In [ ]:
# Visualize orignal Image matrix
print(image)

## 2. Image Preprocessing

### A. Normalize image

In [ ]:
# Before normalization
print("Original Image Shape:", image.shape)

In [ ]:
# Normalization
normalized_image = normalize_image(image)

In [ ]:
# After normalization
print("Normalized Image Shape:", normalized_image.shape)

In [ ]:
normalized_image.dtype

### B. Plot Normalized Image

In [ ]:
print("Min and Max Values after Normalization:", np.min(normalized_image), np.max(normalized_image))
plt.imshow(normalized_image)
plt.show()

In [ ]:
from matplotlib import image as mpimg

In [ ]:
# Assuming normalized_image is a NumPy array in RGB format
mpimg.imsave("images/normalized_fire_false_color_image.jpg", normalized_image)

In [ ]:
print(np.min(normalized_image), np.max(normalized_image))

### C. Normalized Image Matrix

In [ ]:
# Visualize normalized Image matrix
print(normalized_image)

### D. Plot Original Image vs Normalized Image

In [ ]:
### Visualize original vs normalized image

# Display the images using Matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the original image
ax1.imshow(image)
ax1.set_title('Original Image')
ax1.axis('off')

# Plot the normalized image
normalized_image_copy = normalized_image
ax2.imshow(normalized_image_copy)
ax2.set_title('Normalized Image')
ax2.axis('off')

plt.show()

### 2. Anotate the Image

### A.  Overlay anotation on normalized image

In [ ]:
import matplotlib.patches as patches
import json

# Load your image
image = normalized_image.copy()

# Load your annotation data
with open("annotations/anot_norm.json", "r") as file:
    annotation_data = json.load(file)

# Create a figure and axis
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

# Mapping colors to classes
class_colors = {"Fire": 'red', "Smoke": 'purple', "No-Fire": 'green'}

# Overlay each bounding box on the image
for box in annotation_data["boxes"]:
    if box["type"] == "polygon":
        # Extract polygon points
        polygon_points = box["points"]

        # Determine class label
        class_label = box["label"]

        # Define colors based on class
        if class_label == "1":
            edge_color, face_color = 'red', 'none'
        elif class_label == "2":
            edge_color, face_color = 'purple', 'none'
        elif class_label == "3":
            edge_color, face_color = 'green', 'none'
 

        # Create a polygon patch
        polygon = patches.Polygon(polygon_points, linewidth=0.7, edgecolor= edge_color, facecolor=face_color)

        # Add the polygon to the axis
        ax.add_patch(polygon)
# Show legend for classes
ax.legend(handles=[patches.Patch(color=color, label=label) for label, color in class_colors.items()], loc='upper right')
# Show the image with overlay
plt.show()

### B. Plot of Normalized image vs Annotated Image

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json

# Load normalized image

# Loading annotation data
with open("annotations/anot.json", "r") as file:
    annotation_data = json.load(file)

#  a figure and axis with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Display the normalized image
ax1.imshow(normalized_image)
ax1.set_title('Normalized Image')
ax1.axis('off')

# Display the normalized image with annotations
ax2.imshow(normalized_image)

# Mapping colors to classes
class_colors = {"Fire": 'red', "Smoke": 'purple', "No-Fire": 'green'}

# Loop through each bounding box and plot
for box in annotation_data["boxes"]:
    if box["type"] == "polygon":
        # Extract polygon points
        polygon_points = box["points"]

        # Determine class label
        class_label = box["label"]

        # Get color based on class label
        color = class_colors.get(class_label, 'yellow')  # Default to yellow for unknown classes

        # Create a polygon patch
        polygon = patches.Polygon(polygon_points, linewidth=0.8, edgecolor=color, facecolor='none', label=class_label)

        # Add the polygon to the axis
        ax2.add_patch(polygon)

# Show legend for classes
ax2.legend(handles=[patches.Patch(color=color, label=label) for label, color in class_colors.items()], loc='upper right')

ax2.set_title('Annotated Image')
ax2.axis('off')

# Show the plot
plt.show()

### C. Tile image and Pair Anotations

#### C1. Load Anotation

In [ ]:
# Load annotation data
with open("annotations/anot.json", "r") as file:
    annotation_data = json.load(file)

In [ ]:
annotation_data

#### C2. Tiling Function 

In [ ]:
def tile_image(normalized_image, tile_size, overlap):
    """
    Creates image tiles with optional overlap, padding incomplete tiles with zeros.

    Args:
        normalized_image: The normalized image as a NumPy array.
        tile_size: A tuple (height, width) specifying the desired tile size.
        overlap: A tuple (vertical_overlap, horizontal_overlap) specifying the overlap between tiles.

    Returns:
        tiled_images: A list of tiled images as NumPy arrays.
    """

    image = normalized_image  # Make a copy to avoid modifying the original

    height, width = image.shape[:2]
    tile_height, tile_width = tile_size
    vert_overlap, horiz_overlap = overlap

    # Calculate the number of tiles needed, accounting for potential edge cases
    num_vert_tiles = int(np.ceil((height - tile_height) / (tile_height - vert_overlap)) + 1)
    num_horiz_tiles = int(np.ceil((width - tile_width) / (tile_width - horiz_overlap)) + 1)

    tiles = []
    for i in range(num_vert_tiles):
        start_y = i * (tile_height - vert_overlap)
        end_y = min(start_y + tile_height, height) 

        for j in range(num_horiz_tiles):
            start_x = j * (tile_width - horiz_overlap) 
            end_x = min(start_x + tile_width, width) 

            tile = image[start_y:end_y, start_x:end_x]

            # Pad if necessary
            if tile.shape[0] < tile_height:
                pad_y = tile_height - tile.shape[0]
                tile = np.pad(tile, ((0, pad_y), (0, 0), (0, 0)), mode='constant') 
            if tile.shape[1] < tile_width:
                pad_x = tile_width - tile.shape[1]
                tile = np.pad(tile, ((0, 0), (0, pad_x), (0, 0)), mode='constant')

            tiles.append(tile)

    return tiles  

In [ ]:
# Creates 512x512 tiles with 256 pixel overlap
tiles = tile_image(normalized_image.copy(), tile_size=(512, 512), overlap=(0, 0))

#### C3. Print Length of Tile

In [ ]:
len(tiles)

In [ ]:
for tile in tiles:
    print(tile.shape)

#### C4. Print a random tile by specifying the idx number

In [ ]:
### Plot a random tile

plt.imshow(tiles[7])
plt.title('Tile')
plt.axis('off')
plt.show()

#### C5. Plot the first 10 tiles to visualize each tile

In [ ]:
# Tiles is a list containing different tiles
num_rows = 2
num_cols = 5

# Create a subplot grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 6))

# Flatten the 2D array of axes to simplify indexing
axes = axes.flatten()

# Plot each tile
for i in range(num_rows * num_cols):
    # Check if there are still tiles left
    if i < len(tiles):
        axes[i].imshow(tiles[i])
        axes[i].set_title(f'Tile {i + 1}')
        axes[i].axis('off')
    else:
        # If no more tiles, remove the empty subplot
        fig.delaxes(axes[i])

# Adjust layout to prevent clipping of titles
plt.tight_layout()
plt.show()

##### C5.1 Tile Dimention 

In [ ]:
## Tile shape
print(f"Image shape : {tiles[12].shape}")

##### C5.2Visualize Matrix of any random tile, tiles[n] where n = arrIDX

In [ ]:
print(tiles[6])

In [ ]:
print(annotation_data.keys())

In [ ]:
def adjust_and_map_labels(annotation_data, tile_size, overlap, original_image_shape):
    """
    Adjusts annotation labels for image tiles, maps them to a new structure, 
    and includes the tile index for each tile.

    Args:
        annotation_data: The original annotation data as a dictionary.
        tile_size: A tuple (height, width) specifying the tile size.
        overlap: A tuple (vertical_overlap, horizontal_overlap) specifying the overlap between tiles.
        original_image_shape: A tuple (height, width) of the original image.

    Returns:
        A list of dictionaries, each containing adjusted annotations for a specific tile.
    """

    tile_height, tile_width = tile_size
    vert_overlap, horiz_overlap = overlap
    orig_height, orig_width = original_image_shape

    adjusted_annotations = []

    # Calculate tile indices based on original image dimensions
    num_vert_tiles = int(np.ceil((orig_height - tile_height) / (tile_height - vert_overlap)) + 1)
    num_horiz_tiles = int(np.ceil((orig_width - tile_width) / (tile_width - horiz_overlap)) + 1)

    for tile_y in range(num_vert_tiles):
        for tile_x in range(num_horiz_tiles):
            tile_annotations = {'boxes': []}  # Initialize annotations for this tile

            # Calculate tile offsets
            offset_x = tile_x * (tile_width - horiz_overlap)
            offset_y = tile_y * (tile_height - vert_overlap)

            for box in annotation_data['boxes']:
                new_box = box.copy()

                # Adjust coordinates for tile offset
                new_box['x'] = float(box['x']) - offset_x
                new_box['y'] = float(box['y']) - offset_y

                # Adjust points (only for polygon type)
                if box['type'] == 'polygon':
                    new_points = []
                    for point in box['points']:
                        new_points.append([point[0] - offset_x, point[1] - offset_y])
                    new_box['points'] = new_points

                tile_annotations['boxes'].append(new_box)

            # Add tile-specific metadata
            tile_annotations['tile_index'] = (tile_y, tile_x)  

            adjusted_annotations.append(tile_annotations)

    return adjusted_annotations

In [ ]:
tile_size = (512, 512)  # adjusted to match tile size
overlap = (0, 0)    # adjusted to match overlap size
original_image_shape = (annotation_data['height'], annotation_data['width'])

new_annotations = adjust_and_map_labels(annotation_data.copy(), tile_size, overlap, original_image_shape)

In [ ]:
len(new_annotations)

In [ ]:
new_annotations[0:5]

In [ ]:
len(new_annotations)

In [ ]:
def map_labels_to_integers(new_annotations):
    """
    Maps string labels in annotations to integer codes and converts string-based
    'width' and 'height' to numeric types (floats).

    Args:
        new_annotations: A list of dictionaries where each dictionary represents
                         annotations for a tile.

    Returns:
        The same list of annotation dictionaries with labels mapped to integers 
    """

    label_mapping = {
        "Fire": 1,
        "Smoke": 2,
        "No-Fire": 3
    }

    for tile_annotations in new_annotations:
        for box in tile_annotations['boxes']:
            box['label'] = label_mapping[box['label']]
    return new_annotations


In [ ]:
mapped_annotations = map_labels_to_integers(new_annotations.copy())  # Created a copy to be safe

In [ ]:
# View mapped annotations
mapped_annotations

In [ ]:
mapped_annotations[0]

### D. Training and Validation Split 

#### D1. Convert labels and tiles to NumPy arrays

In [ ]:
## Convert tiles to np array
tiles_array = np.stack([np.array(tile) for tile in tiles])

In [ ]:
len(tiles_array)

In [ ]:
type(tiles_array)

In [ ]:
### Plot a random tile

plt.imshow(tiles_array[7])
plt.title('Tile')
plt.axis('off')
plt.show()

In [ ]:
type(tiles_array)

In [ ]:
def convert_annotations_to_arrays(mapped_annotations):
    all_arrays = []
    for annotation in mapped_annotations:
        points = annotation['boxes'][0]['points']
        points_array = np.array(points)
        all_arrays.append(points_array)
    return np.array(all_arrays)

In [ ]:
label_array = convert_annotations_to_arrays(mapped_annotations)

In [ ]:
type(label_array)

In [ ]:
len(label_array)

In [ ]:
print(f'Shape of tiles converted to numpy array {tiles_array.shape}')
print(f'Shape of labels converted to numpy array {label_array.shape}')

In [ ]:
# create blank tiles for labelling 
def create_segmentation_mask(tile, annotations):
    mask = np.zeros((512, 512), dtype=np.uint8)  # Blank mask

    # Assuming 'annotations' is a dictionary representing a single tile's annotations
    for box in annotations['boxes']: 
        points = np.array(box['points'], dtype=np.int32)
        cv2.fillPoly(mask, [points], color=box['label'])  
    return mask # Return the created mask

# Outside the `create_segmentation_mask` function:
label_array = [] 
for i in range(len(tiles_array)):
    tile = tiles_array[i] 
    tile_annotations = mapped_annotations[i] 
    mask = create_segmentation_mask(tile, tile_annotations) 
    label_array.append(mask) # Append the mask for the current tile

In [ ]:
len(label_array)

In [ ]:
type(label_array)

In [ ]:
# Convert label_array to a NumPy array
label_array_np = np.array(label_array)

# Create an empty array with the desired shape (72, 512, 512, 3)
expanded_label_array = np.zeros((label_array_np.shape[0], label_array_np.shape[1], label_array_np.shape[2], 3))

# Assign the original label array to the first channel
expanded_label_array[:, :, :, 0] = label_array_np

# Check the shape of the expanded label array
print(f'Expanded label array shape: {expanded_label_array.shape}')

In [ ]:
print(f'Label array {np.array(expanded_label_array).shape}')
print(f'Tiles array {np.array(tiles_array).shape}')

#### D2. Split Operation

In [ ]:
# Import library
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(tiles_array, expanded_label_array, test_size=0.3, random_state=42)

#### D3. Confirm Split

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

In [ ]:
# check data types
print(f'Type of X_train {type(X_train)}')
print(f'Type of X_val {type(X_val)}')
print(f'Type of y_train {type(y_train)}')
print(f'Type of y_val {type(y_val)}')

In [ ]:
print(f'X_train {X_train[3]}')
print(f'X_val {X_val[3]}')
print(f'y_train {y_train[3]}')
print(f'y_val {y_val[3]}')

#### Note:
<p> 
    - X_train and y_train contains the training data <br> 
    - X_val and y_val contains the validation data
</p>

## 3. UNET Model

### A. Import Libraries 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### B. SetUp Early Stopping 

In [ ]:
# Define directory to save the checkpoints
checkpoint_dir = 'checkpoints/'

# directory check
os.makedirs(checkpoint_dir, exist_ok=True)

# checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_checkpoint.h5'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Define the early stopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

### C. UNET Model Design

In [ ]:
def unet():
    inputs = Input((512, 512, 3))

    # Encoder
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)  # Reduce filter count
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)  # Reduce filter count
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)  # Reduce filter count
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up3 = UpSampling2D(size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation='relu', padding='same')(up3)  # Reduce filter count
    conv4 = Conv2D(64, 3, activation='relu', padding='same')(conv4)

    merge1 = concatenate([conv2, conv4], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge1)  # Reduce filter count
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    up2 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(32, 3, activation='relu', padding='same')(up2)  # Reduce filter count
    conv6 = Conv2D(32, 3, activation='relu', padding='same')(conv6)

    merge2 = concatenate([conv1, conv6], axis=3)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(merge2)  # Reduce filter count
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(conv7)

    # Output layer (modify based on your number of classes)
    outputs = Conv2D(3, 1, activation='softmax', padding='same')(conv7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


### D. Compile Model

In [ ]:
# Compile the model with an appropriate loss function for multiclass segmentation
unet_model = unet()
unet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### E. Model Summary

In [ ]:
unet_model.summary()

### F. Fit Model

In [ ]:
unet_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
#     callbacks=[checkpoint_callback, early_stopping_callback],
#     verbose=2
)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(tiles_array, expanded_label_array, test_size=0.3)

In [ ]:
score = unet_model.evaluate(test_data, test_labels, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#get first 10 images, view input and ouput images on every line
test_preds = unet_model.predict(test_data[0:8])

In [ ]:
#test
print(test_preds.shape)  # Check the shape
print(test_preds.dtype)  # Check the data type

In [ ]:
predictions = unet_model.predict(test_data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_predictions(images, predictions, class_names):
    """
    Visualizes all test images side-by-side with their dominant fire detection labels.

    Args:
        images: A NumPy array of the test images.
        predictions: A NumPy array of the predicted masks (argmax for class index).
        class_names: A list of class names corresponding to the label indices.
    """
    num_images = images.shape[0]
    num_rows = int(np.ceil(num_images / 5))  # Adjust columns for better layout
    num_cols = min(5, num_images)  # Show max 5 images per row

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 5))

    # Flatten axes for easier iteration
    axes_flat = axes.ravel()

    for i in range(num_images):
        image = images[i]
        prediction = predictions[i].flatten()

        # Check if fire is present based on predicted probabilities
        threshold = 0.7  # Adjust the threshold as needed
        if np.max(prediction) > threshold:
            fire_detected = class_names[np.argmax(prediction)]
        else:
            fire_detected = "No Fire Detected"

        class_text = fire_detected

        # Plot image and title
        axes_flat[i].imshow(image)
        axes_flat[i].set_title(class_text)
        axes_flat[i].axis('off')

    # Hide extra axes if fewer images than total plots
    for i in range(num_images, num_rows * num_cols):
        axes_flat[i].axis('off')

    plt.tight_layout()
    plt.show()

# Assuming you have test_data and test_preds
# Get class names (assuming you have them)
class_names = ["Fire", "Smoke", "No-Fire"]  # Replace with your actual class names

# Visualize all images
visualize_predictions(test_data, test_preds.argmax(axis=-1), class_names)